In [2]:
import pandas as pd 
import numpy as np
import time 
import random
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [4]:
driver=webdriver.Chrome()
driver.get('https://serieson.naver.com/v3/movie/products?tag=&sortType=UPDATE_DESC&price=all')

# options = webdriver.ChromeOptions()
# options.add_argument('--disable-dev-shm-usage')
# service = Service(executable_path="./chromedriver.exe")
# driver = webdriver.Chrome(service=service,options=options)
# driver.get("https://serieson.naver.com/v3/movie/products?tag=&sortType=POPULARITY_DESC&price=all")

driver.implicitly_wait(10)
time.sleep(3)

In [63]:
# 내용 가져오기
total={}

for _ in range(10):
    main=driver.find_elements(By.CSS_SELECTOR, ".ExpandedListCollection_item___Swc_")

    for i in tqdm(main):

        result = i.get_attribute("innerHTML")
        from bs4 import BeautifulSoup
        bs = BeautifulSoup(result,"html.parser")

        try:
            name = bs.select_one('.ExpandedPoster_title__P7ff9').text
            star = bs.select_one('.ExpandedPoster_info_item__ZsGjd').text.split('\n')[-1]
            director = bs.select_one('.ExpandedPoster_info_item__ZsGjd.ExpandedPoster_directors__JBSE1').text
            actor = bs.select_one('.ExpandedPoster_info_item__ZsGjd.ExpandedPoster_actors__1lZnU').text
            date_ = bs.select('.ExpandedPoster_info_item__ZsGjd')[-1].text
            content=bs.select_one('.ExpandedPoster_synopsis__KyuZ7').text

            total[name]=[star,director,actor,date_,content]
        except:
            pass
    
    try:
        driver.find_element(By.CSS_SELECTOR, '#content > div.ListPage_view_more_button_wrapper__2LsYx > button').click()
        time.sleep(1+random.random()*2)
    except:
        break

100%|██████████| 300/300 [00:03<00:00, 90.61it/s]


In [5]:
# '더보기' 클릭
for _ in tqdm(range(220)):
    try:
        driver.find_element(By.CSS_SELECTOR, 
                            '#content > div.ListPage_view_more_button_wrapper__2LsYx > button').click()
        time.sleep(1 + random.random())
    except:
        break

  0%|          | 0/220 [00:00<?, ?it/s]100%|██████████| 220/220 [24:16<00:00,  6.62s/it]


In [6]:
# 전체 셀레늄 사용 시 속도가 느림 => 셀레늄으로 불러온 후 bs로 각 속성의 텍스트 추출
# 중복된 영화(대여/구매)가 있어서 dict type { 영화제목 : [평점, 감독, 배우, 개봉일자, 줄거리] } 형식으로 저장

total={}

main=driver.find_elements(By.CSS_SELECTOR, ".ExpandedListCollection_item___Swc_")

for i in tqdm(main):
    result = i.get_attribute("innerHTML")
    bs = BeautifulSoup(result,"html.parser")

    try:
        name = bs.select_one('.ExpandedPoster_title__P7ff9').text
        star = bs.select_one('.ExpandedPoster_info_item__ZsGjd').text.split('\n')[-1]
        director = bs.select_one('.ExpandedPoster_info_item__ZsGjd.ExpandedPoster_directors__JBSE1').text
        actor = bs.select_one('.ExpandedPoster_info_item__ZsGjd.ExpandedPoster_actors__1lZnU').text
        date_ = bs.select('.ExpandedPoster_info_item__ZsGjd')[-1].text
        content=bs.select_one('.ExpandedPoster_synopsis__KyuZ7').text

        total[name]=[star,director,actor,date_,content]
    except:
        pass

100%|██████████| 6630/6630 [02:12<00:00, 49.94it/s]


In [19]:
# Dict -> List로 저장
result = [[k,v[0],v[1],v[2],v[3],v[4]] for k,v in total.items()]

In [20]:
# DF로 저장
result=pd.DataFrame(result,columns=['제목','평점','감독','배우','개봉일자','줄거리'])

In [28]:
# 제거 :  ","  "\n"  "\r"   
#       "감독" "별점" "주연"
result = result.applymap(lambda x : x.replace(",",""))
result = result.applymap(lambda x : x.replace("\n",""))
result = result.applymap(lambda x : x.replace("\r",""))
result['감독']=result['감독'].apply(lambda x : x.replace("감독",""))
result['평점']=result['평점'].apply(lambda x : x.replace("별점",""))
result['배우']=result['배우'].apply(lambda x : x.replace("주연",""))
result

,제목,평점,감독,배우,개봉일자,줄거리
0,범죄도시3,7.6,이상용,마동석 이준혁 아오키 무네타카 이범수 김민재 이지훈 김도건 고규필 전석호 안세호,2023.05.31.,대체불가 괴물형사 마석도 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 뒤...
1,엘리멘탈(패키지: 자막판+더빙판+부가영상+코멘터리),9.4,피터 손,레아 루이스 마무두 아티,2023.06.14.,불 물 공기 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼 열정...
2,공조2: 인터내셔날,8.3,이석훈,현빈 유해진 윤아 다니엘 헤니 진선규 장영남 박훈 임성재 윤상화 박민하,2022.09.07.,공조 이즈 백! 이번엔 삼각 공조다!남한으로 숨어든 글로벌 범죄 조직을 잡기 위해 ...
3,아바타: 물의 길(부가영상 제공),8.6,제임스 카메론,조 샐다나 샘 워싱턴 시고니 위버 우나 채플린 지오바니 리비시 스티븐 랭 케이트 ...,2022.12.14.,<아바타: 물의 길>은 판도라 행성에서 '제이크 설리'와 '네이티리'가 이룬 가족이...
4,올빼미,9.3,안태진,류준열 유해진 최무성 조성하 박명훈 김성철 안은진 조윤서 이주원 김예은,2022.11.23.,맹인이지만 뛰어난 침술 실력을 지닌 ‘경수’는어의 ‘이형익’에게 그 재주를 인정받아...
...,...,...,...,...,...,...
5191,트론: 새로운 시작,7.3,조셉 코신스키,제프 브리지스 가렛 헤드룬드 올리비아 와일드 마이클 쉰 애니스 처파 브루스 박스라...,2010.12.29.,세상 누구도 본적 없는 최고의 가상 현실을 창조한 천재박사 케빈 플린(제프 브리지스...
5192,리버티 벨런스를 쏜 사나이,8.4,존 포드,존 웨인 제임스 스튜어트 베라 마일즈 리 마빈 에드먼드 오브라이언 앤디 데빈 켄 ...,주연 존 웨인 제임스 스튜어트 베라 마일즈 리 마빈 에드먼드 오브라이언 앤디 데빈 ...,랜스(제임스 스튜어트 분)는 자신의 오래전 친구 톰(존 웨인 분)의 장례식에 참석하...
5193,리썰 웨폰,8.3,리차드 도너,멜 깁슨 대니 글로버 게리 부시 미치 라이언 톰 앳킨스 달린 러브,1987.12.01.,술집 여종업원인 아멘다 훤세커(Amanda Hunsaker: 잭키 스완슨 분)가 약...
5194,몽골리안 프린세스,7.5,정단우,정단우 엘리자베스 가르시아 박하나,2015.03.19.,34살이 되도록 연애 한번 못해보고 영화밖에 모르고 살아온 ‘단우’. 어느 날 본인...


In [32]:
# csv 파일로 저장
result.to_csv("movie.csv",encoding='utf-8',index=False)